In [3]:
!poetry run pip install pandas

  Obtaining dependency information for pandas from https://files.pythonhosted.org/packages/f8/7f/5b047effafbdd34e52c9e2d7e44f729a0655efafb22198c45cf692cdc157/pandas-2.0.3-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Obtaining dependency information for tzdata>=2022.1 from https://files.pythonhosted.org/packages/65/58/f9c9e6be752e9fcb8b6a0ee9fb87e6e7a1f6bcab2cdc73f02bb7ba91ada0/tzdata-2024.1-py2.py3-none-any.whl.metadata
  Obtaining dependency information for numpy>=1.20.3 from https://files.pythonhosted.org/packages/98/5d/5738903efe0ecb73e51eb44feafba32bdba2081263d40c5043568ff60faf/numpy-1.24.4-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 36.0 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 34.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.4/345.4 kB 24.9 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.2 -

In [4]:
from collections import defaultdict
from django.db.models.signals import *
import pandas as pd
from itertools import chain
from tqdm.notebook import tqdm
import datetime
import calendar

In [5]:
def normalize_month(month, year):
    return month + 12*(year - 2020)

In [6]:
normalize_month(12, 2023)

48

In [7]:
# def normalize_day(day, month):
#     return day + 31*(month - 1)

In [8]:
# normalize_day(31,12)

In [9]:
def permissoes_history_user(result_list,mes_ano):
    permissoesUsuarios = {}
    mes_referencia = normalize_month(mes_ano.month, mes_ano.year)
    for a in result_list:
        try:
            if not permissoesUsuarios.get(a.user.get_full_name()):
                permissoesUsuarios[a.user.get_full_name()] = {}
            if isinstance(permissoesUsuarios.get(a.user.get_full_name()).get(normalize_month(a.history_date.month, a.history_date.year)),list):
                permissoesUsuarios[a.user.get_full_name()][normalize_month(a.history_date.month, a.history_date.year)].append(a.permissions.name)
            elif permissoesUsuarios.get(a.user.get_full_name()).get(normalize_month(a.history_date.month, a.history_date.year)):
                multi_perm_per_month= permissoesUsuarios.get(a.user.get_full_name()).get(normalize_month(a.history_date.month, a.history_date.year))
                permissoesUsuarios[a.user.get_full_name()][normalize_month(a.history_date.month, a.history_date.year)] = [multi_perm_per_month,a.permissions.name]
            else:
                permissoesUsuarios[a.user.get_full_name()][normalize_month(a.history_date.month, a.history_date.year)] = a.permissions.name
        except:
            pass
    
    usuariosAtivos = []
    usuariosInativos = []

    for nome, mes in permissoesUsuarios.items():
        meses = sorted(list(mes.keys()))
        item = max(list(mes.keys()))
        cargo_atual = mes[item]

# Se em algum momento do mesmo mês o usuário foi diferente de Inativo ele entra como ativo
        if isinstance(cargo_atual,list):
            for cargo in cargo_atual:
                if cargo !='Inativo' and not any(u for u in usuariosAtivos if nome == u):
                    usuariosAtivos.append(nome)     
# se o cargo atual for diferente de inativo e não for uma lista(o que configura multiplas permissões no mesmo mês) ele é ativo
        elif cargo_atual != 'Inativo' and not isinstance(cargo_atual,list):
            usuariosAtivos.append(nome)
# Se no mes anterior ao ultimo ele foi diferente de inativo e no ultimo mes foi Inativo e
# o mes do cargo_atual for igual ao ultimo mes de referencia(ultimo mês pra coleta) e o nome não estiver nos usuariosAtivos
# ele é ativo
        elif mes[meses[-2]] != 'Inativo' and cargo_atual == 'Inativo' and item == mes_referencia and not any(u for u in usuariosAtivos if nome == u):
            usuariosAtivos.append(nome)       
# se o cargo atual contem a palavra Inativo e não está nos usuariosAtivos então ele é Inativo
        elif 'Inativo' in cargo_atual and nome not in usuariosAtivos:
            usuariosInativos.append(nome)
        else:
            print('Corra pras montanhas')
    
    return usuariosAtivos,usuariosInativos

In [6]:
companies = ['Modera - Consórcio ME2', 
             'Sinalizavia (Municípios)', 
             'Afirma Engenharia Viária',
             'Pavicolet', 
             'Sanson',
             'Entrevias',
             'Televale',
             'Garcia Monteiro',
             'LCM Construção e Comércio',
             'Construtora Castilho']

len(companies)

10

In [11]:
# Mês e ano para coleta dos usuários
mes=3
ano=2024
dia = calendar.monthrange(ano, mes)[1]
dia,mes,ano

(31, 3, 2024)

In [12]:
# DATA sempre referente ao mês que se quer coletar nesse formato history_date__lte='2024-01-31 03:00:00'
# é utilizado 3 horas da manha pois é a pra fechar com o horario de meio noite do servidor
data = datetime.datetime(ano,mes,dia,3,0,0)
data

datetime.datetime(2024, 3, 31, 3, 0)

In [13]:
pd.DataFrame().to_excel('Usuários Ativos Financeiro.xlsx')

In [14]:
for company in tqdm(companies):
    if 'Pavicolet' in company:
        historicoUsuarios=UserInCompany.history.model.objects.filter(
            company__name__icontains='Pavicolet',
            history_date__lte=data
        ).exclude(
            user__email__icontains='kartado'
        ).exclude(
            user__email__icontains='roadlabs'
        ).exclude(
            user__uuid = '6ca2ac08-7ee0-4587-a0cf-8850f72e1b95'
        ).exclude(
            user__email__icontains='hermes'
        ).order_by( 'history_date')
    elif company is not 'Entrevias':
        historicoUsuarios=UserInCompany.history.model.objects.filter(
            company__name=company,
            history_date__lte=data
        ).exclude(
            user__email__icontains='kartado'
        ).exclude(
            user__email__icontains='roadlabs'
        ).exclude(
            user__uuid = '6ca2ac08-7ee0-4587-a0cf-8850f72e1b95'
        ).exclude(
            user__email__icontains='hermes'
        ).order_by( 'history_date')
    else:
        historicoUsuarios=UserInCompany.history.model.objects.filter(
            Q(user__email__icontains='afirma')|Q(user__email__icontains='ajr'),
            company__name__icontains=company,
            history_date__lte=data
        ).exclude(
            user__email__icontains='kartado'
        ).exclude(
            user__email__icontains='roadlabs'
        ).exclude(
            user__uuid = '6ca2ac08-7ee0-4587-a0cf-8850f72e1b95'
        ).exclude(
            user__email__icontains='hermes'
        ).order_by( 'history_date')
    
    users_permissions = permissoes_history_user(historicoUsuarios,data)[0] if len(permissoes_history_user(historicoUsuarios,data)[0]) > 0 else 0
    
    if users_permissions != 0:
        df2 = pd.DataFrame(data=users_permissions,columns=['Usuários ativos'])
        with pd.ExcelWriter('Usuários Ativos Financeiro.xlsx', mode='a',if_sheet_exists='replace') as writer:
            df2.to_excel(writer,sheet_name=company.replace('/','')[:15],na_rep="-")

<>:15: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:15: SyntaxWarning: "is not" with a literal. Did you mean "!="?
/tmp/ipykernel_7665/2874654108.py:15: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  elif company is not 'Entrevias':


  0%|          | 0/14 [00:00<?, ?it/s]

/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.8/lib/python3.8/site-packages/django/db/models/fields/__init__.py:1424: RuntimeWarning: DateTimeField HistoricalUserInCompany.history_date received a naive datetime (2024-03-31 03:00:00) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"


In [10]:
companies=['Neovia Alcinópolis',
'Neovia Araquari 886/2020 ',
'Neovia Araranguá',
'Neovia Bocaína do Sul 329/2018',
'Neovia Contorno Sul ',
'Neovia Cto Biguaçu',
'Neovia Diamantino',
'Neovia Lages 652/2019',
'Neovia Lagoa Vermelha',
'Neovia Planalto Sul ',
'Neovia Ponte Serrada',
'Neovia Rancho Queimado 080/2021',
'Neovia Régis Bittencourt',
'Neovia Tijucas',]

# TESTES

In [78]:
# def permissoes_history_user_v2(result_list,mes_ano):
#     permissoesUsuarios = {}
#     mes_referencia = normalize_month(mes_ano.month, mes_ano.year)
#     for a in result_list:
#         try:
# #             print(a.user.get_full_name())
#             if not permissoesUsuarios.get(a.user.get_full_name()):
#                 permissoesUsuarios[a.user.get_full_name()] = {}
#             if isinstance(permissoesUsuarios.get(a.user.get_full_name()).get(normalize_month(a.history_date.month, a.history_date.year)),list):
#                 permissoesUsuarios[a.user.get_full_name()][normalize_month(a.history_date.month, a.history_date.year)].append(a.permissions.name)
#             elif permissoesUsuarios.get(a.user.get_full_name()).get(normalize_month(a.history_date.month, a.history_date.year)):
#                 multi_perm_per_month= permissoesUsuarios.get(a.user.get_full_name()).get(normalize_month(a.history_date.month, a.history_date.year))
#                 permissoesUsuarios[a.user.get_full_name()][normalize_month(a.history_date.month, a.history_date.year)] = [multi_perm_per_month,a.permissions.name]
#             else:
#                 permissoesUsuarios[a.user.get_full_name()][normalize_month(a.history_date.month, a.history_date.year)] = a.permissions.name
#         except:
#             pass
    
#     usuariosAtivos = []
#     usuariosInativos = []
#     print(permissoesUsuarios)
#     for nome, mes in permissoesUsuarios.items():
#         meses = sorted(list(mes.keys()))
#         item = max(list(mes.keys()))
#         cargo_atual = mes[item]
#         print(nome,mes)
# # Se em algum momento do mesmo mês o usuário foi diferente de inativo ele entra como ativo
#         if isinstance(cargo_atual,list):
#             for cargo in cargo_atual:
#                 if cargo !='Inativo' and not any(u for u in usuariosAtivos if nome == u):
#                     usuariosAtivos.append(nome)     
# # se o cargo atual for diferente de inativo e não for uma lista(o que configura multiplas permissões no mesmo mês) ele é ativo
#         elif cargo_atual != 'Inativo' and not isinstance(cargo_atual,list):
#             usuariosAtivos.append(nome)
# # Se no mes anterior ao ultimo ele foi diferente de inativo e no ultimo mes foi Inativo e
# # o mes do cargo_atual for igual ao ultimo mes de referencia(ultimo mês pra coleta) e o nome não estiver nos usuariosAtivos
# # ele é ativo
#         elif mes[meses[-2]] != 'Inativo' and cargo_atual == 'Inativo' and item == mes_referencia and not any(u for u in usuariosAtivos if nome == u):
#             usuariosAtivos.append(nome)
# # se o cargo atual contem a palavra Inativo e não está nos usuariosAtivos então ele é Inativo
#         elif 'Inativo' in cargo_atual and nome not in usuariosAtivos:
#             usuariosInativos.append(nome)
#         else:
#             print('Corra pras montanhas')
    
#     return usuariosAtivos,usuariosInativos

In [118]:
# historicoUsuarios=UserInCompany.history.model.objects.filter(
#             company__name='Afirma Engenharia Viária',
#             history_date__lte=data
#         ).exclude(
#             user__email__icontains='kartado'
#         ).exclude(
#             user__email__icontains='roadlabs'
#         ).exclude(
#             user__uuid = '6ca2ac08-7ee0-4587-a0cf-8850f72e1b95'
#         ).exclude(
#             user__email__icontains='hermes'
#         ).order_by( 'history_date')


In [119]:
# a,i=permissoes_history_user_v2(historicoUsuarios,data)

{'Gabriel Pezzi': {25: 'Administrador', 41: 'Inativo', 45: 'Inativo', 46: 'Inativo'}, 'Gerson Bueno': {26: 'Encarregado', 45: 'Inativo', 46: 'Inativo'}, 'Lucas Dadalt': {27: 'Encarregado', 41: 'Inativo', 45: 'Inativo', 46: 'Inativo'}, 'Edson Peicho': {27: ['Encarregado', 'Encarregado'], 32: 'Inativo', 45: 'Inativo', 46: 'Inativo'}, 'Eduardo Henrique': {28: 'Encarregado', 45: 'Inativo', 46: 'Inativo'}, 'Fernando Stipp': {29: 'Administrador', 32: 'Inativo', 45: 'Inativo', 46: 'Inativo'}, 'Tablet 1 Eco 101': {29: 'Encarregado', 32: 'Inativo', 45: 'Inativo', 46: 'Inativo'}, 'Tablet 2 Eco 101': {29: 'Encarregado', 32: 'Inativo', 45: 'Inativo', 46: 'Inativo'}, 'Tablet 3 Eco 101': {29: 'Encarregado', 32: 'Inativo', 45: 'Inativo', 46: 'Inativo'}, 'Tablet 4 Eco 101': {29: 'Encarregado', 32: 'Inativo', 45: 'Inativo', 46: 'Inativo'}, 'Tablet 5 ECO101': {29: 'Encarregado', 32: 'Inativo', 45: 'Inativo', 46: 'Inativo'}, 'Igor Cortes': {29: ['Encarregado', 'Administrador'], 32: 'Inativo', 35: 'Admini

In [120]:
# len(a),len(i),len(a)+len(i)

(3, 18, 21)

In [121]:
# a,i

(['Norton Antunes', 'Jorge Malucelli', 'Elder Santos'],
 ['Gabriel Pezzi',
  'Gerson Bueno',
  'Lucas Dadalt',
  'Edson Peicho',
  'Eduardo Henrique',
  'Fernando Stipp',
  'Tablet 1 Eco 101',
  'Tablet 2 Eco 101',
  'Tablet 3 Eco 101',
  'Tablet 4 Eco 101',
  'Tablet 5 ECO101',
  'Igor Cortes',
  'Stephany Camile',
  'João Alexandre',
  'Mirailton Visitação da Silva',
  'Marcos Silva',
  'Karine Rodrigues',
  'Vinicíus Coelho'])

In [1]:
UserInCompany._meta.fields

(<django.db.models.fields.UUIDField: uuid>,
 <django.db.models.fields.related.ForeignKey: user>,
 <django.db.models.fields.related.ForeignKey: company>,
 <django.db.models.fields.DateField: expiration_date>,
 <django.db.models.fields.IntegerField: level>,
 <django.db.models.fields.related.ForeignKey: permissions>,
 <django.contrib.postgres.fields.array.ArrayField: added_permissions>,
 <django.db.models.fields.BooleanField: is_active>)

In [1]:
UserInCompany.objects.filter(is_active=False)

ProgrammingError: column companies_userincompany.is_active does not exist
LINE 1: ...", "companies_userincompany"."added_permissions", "companies...
                                                             ^
